# Introduction to Remote Sensing with Python

>"...the time is now right and urgent to apply space technology towards the solution of many pressing natural resources problems being compounded by population and industrial growth."

- [Stewart Udall, Secretary of the Interior, September 21, 1966](https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/1966.09.21-DOI-Udall-Earth%20Resources%20Studied%20From%20Space.pdf)

## A remote sensing crash course

<img src="images/remote.png" width=600>

https://youtu.be/DGE-N8_LQBo


## Landsat

Landsat is the first "civilian Earth observation satellite" launched in 1972, a collaboration between the Department of the Interior, NASA, and the Department of Agriculture.

At over 40 years, the Landsat series of satellites provides the longest temporal record of moderate resolution multispectral data of the Earth’s surface on a global basis. The Landsat record has remained remarkably unbroken, proving a unique resource to assist a broad range of specialists in managing the world’s food, water, forests, and other natural resources for a growing world population. It is a record unmatched in quality, detail, coverage, and value. Source: USGS

- [NASA's Landsat page](https://www.nasa.gov/mission_pages/landsat/overview/index.html)
- [USGS's Landsat page](https://www.usgs.gov/core-science-systems/nli/landsat)

<img src="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/full_width/public/thumbnails/image/5262-Landsat-timeline-FNL-2.jpg">

## How to access landsat data

- [USGS Earth Explorer](https://earthexplorer.usgs.gov/)
- [Amazon AWS Open Data](https://registry.opendata.aws/landsat-8/)
- [Google Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/landsat-8)

## Finding satellite image of the 2018 Camp Fire 

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Camp_Fire_oli_2018312_Landsat.jpg/800px-Camp_Fire_oli_2018312_Landsat.jpg">

The Camp Fire was the deadliest and most destructive wildfire in California's history, and the most expensive natural disaster in the world in 2018 in terms of insured losses.

- https://en.wikipedia.org/wiki/Camp_Fire_(2018)

### Google Earth Preview

Before we begin our Python exploration, let us situate ourselves with Google Earth.

<img src="images/camp.png">

<a href="Camp.kmz">Google Earth File</a>

### The Workflow

In this lab, we will do the following:

- Use Google Earth Engine's Python API
- define an AOI (area of interest) in the Northern California's Butte County
- import multiple Landsat images from the dates before and after the Camp Fire
- determine which images are best for analysis
- create various NDVI map outputs to assess the amount of fire damage

## Python libraries in this workshop

- [pandas](https://pandas.pydata.org/docs/getting_started/index.html)
- [geopandas](https://geopandas.org/en/stable/gallery/index.html)
- [google earth engine](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)


## Import libraries

In [3]:
# the regulars
import pandas as pd
import geopandas as gpd

# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

## Authenticate Earth Engine

In [4]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AWtgzh7XSFlqsik2D1PLqbdOEiU8_4ACxmT8RVVe9a9D8-u3t1ApZ0dTa-c

Successfully saved authorization token.


For this lab, we will use Google Earth Engine's "USGS Landsat 8 Level 2, Collection 2, Tier 1"

- [EE Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

You can also:

- `.filterBounds()` method allows you to filter by location
- `.filterDate()` allows you to filter by date

## Define filters

In [28]:
# coordinates of the Camp Fire
lat = 45.3963
lon = 59.6134

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)

# start date of range to filter for
start_date = '2014-10-01'

# end date
end_date = '2020-01-31'

## Get Landsat 8 data

In [29]:
# get the satellite data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)

In [30]:
# how many images did we get?
print('Total number:', landsat.size().getInfo())

Total number: 193


In [31]:
# information about the first image in our collection
landsat.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7651, 7771],
   'crs': 'EPSG:32640',
   'crs_transform': [30, 0, 613485, 0, -30, 5217615]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7651, 7771],
   'crs': 'EPSG:32640',
   'crs_transform': [30, 0, 613485, 0, -30, 5217615]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7651, 7771],
   'crs': 'EPSG:32640',
   'crs_transform': [30, 0, 613485, 0, -30, 5217615]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7651, 7771],
   'crs': 'EPSG:32640',
   'crs_transform': [30, 0, 613485, 0, -30, 5217615]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
 

In [32]:
# what about cloud cover of our first image?
landsat.first().get('CLOUD_COVER').getInfo()

5.21

In [33]:
# when was this image taken?
landsat.first().get('DATE_ACQUIRED').getInfo()

'2014-10-16'

## Bands
<img src="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/thumbnails/image/Landsat%208%20band%20designations.jpg" width=800>

[Source: USGS](https://www.usgs.gov/media/images/landsat-8-band-designations)


<img src="images/bands.jpg" width=1000>

[Source: Satellite Imaging Corporation](https://www.satimagingcorp.com/satellite-sensors/worldview-3/)

In [34]:
# what bands did we get?
landsat.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [35]:
# put the images in a list
landsat_list = landsat.toList(landsat.size());

## Display satellite image

- [What are the min/max values?](https://gis.stackexchange.com/questions/304180/what-are-the-min-and-max-values-of-map-addlayer-on-google-earth-engine)

In [36]:
# set some parameters for the images
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

In [45]:
# create an empty data container
data = []

# loop through each image and display it
for i in range(landsat.size().getInfo()):
    


    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    if(cloud<1):    
        # print the image info
        print('Image #',i,date,'Cloud cover:',cloud)
    
        # display the image
        display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

        # data to list
        this_data = [i,date,cloud]

        # append the data 
        data.append(this_data)
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 1 2014-11-01 Cloud cover: 0.09


Image # 5 2015-02-21 Cloud cover: 0


Image # 29 2016-05-30 Cloud cover: 0.51


Image # 31 2016-07-01 Cloud cover: 0.9


Image # 32 2016-07-17 Cloud cover: 0.15


...

Image # 165 2018-06-05 Cloud cover: 0.28


Image # 170 2018-08-24 Cloud cover: 0.7


Image # 182 2019-06-08 Cloud cover: 0.04


Image # 183 2019-06-24 Cloud cover: 0.01


Image # 188 2019-09-12 Cloud cover: 0.86


In [46]:
# our data in a dataframe
df

,Image #,Date,Cloud Cover
0,1,2014-11-01,0.09
1,5,2015-02-21,0.00
2,29,2016-05-30,0.51
3,31,2016-07-01,0.90
4,32,2016-07-17,0.15
5,33,2016-08-02,0.03
6,42,2017-02-10,0.01
7,51,2017-07-04,0.27
8,53,2017-08-05,0.00
9,54,2017-08-21,0.69


## Selecting images, zooming in
Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study. Ideally, we want to have images for before and after the fire to be able to assess the level of damage.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by appying a 20km buffer around our POI.

In [53]:
# # create a list of images we want (before, during, after)
landsat_sequence = df['Image #'].tolist()

In [54]:
# Define a region of interest with a buffer zone of 20 km
roi = poi.buffer(20000) # meters

In [55]:
parameters = {
                'min': 6000,
                'max': 16000,
                'dimensions': 800,
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                'region':roi
             }

In [56]:
for i in landsat_sequence:
    
    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = ee.Image(landsat_list.get(i)).get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = ee.Image(landsat_list.get(i)).getThumbUrl(parameters)))

Image # 1 2014-11-01 Cloud cover: 0.09


Image # 5 2015-02-21 Cloud cover: 0


Image # 29 2016-05-30 Cloud cover: 0.51


Image # 31 2016-07-01 Cloud cover: 0.9


Image # 32 2016-07-17 Cloud cover: 0.15


...

Image # 165 2018-06-05 Cloud cover: 0.28


Image # 170 2018-08-24 Cloud cover: 0.7


Image # 182 2019-06-08 Cloud cover: 0.04


Image # 183 2019-06-24 Cloud cover: 0.01


Image # 188 2019-09-12 Cloud cover: 0.86


<div class="alert alert-info">
Take a moment to inspect the three images above. What do they tell you? What do they NOT tell you?
</div>

## Normalized Difference Vegetation Index (NDVI)

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

<img src="images/ndvi.png" width=600>

- [Source: Agricolus](https://www.agricolus.com/en/vegetation-indices-ndvi-ndmi/)

### Calculating NDVI in Google Earth Engine

- https://developers.google.com/earth-engine/tutorials/tutorial_api_06

In [57]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [58]:
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()
    
    # print some information
    print('Image #',i,date)
    
    # display the image
    display(Image(url=ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

Image # 1 2014-11-01


Image # 5 2015-02-21


Image # 29 2016-05-30


Image # 31 2016-07-01


Image # 32 2016-07-17


...

Image # 165 2018-06-05


Image # 170 2018-08-24


Image # 182 2019-06-08


Image # 183 2019-06-24


Image # 188 2019-09-12


## Folium

Of course, we can't end the lab without an interactive map. For this, we can use [folium](https://python-visualization.github.io/folium/quickstart.html), which uses the open-source javascript mapping library [leaflet](https://leafletjs.com/).

- https://python-visualization.github.io/folium/quickstart.html

Google earth engine works with folium:
- https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard?hl=en#interactive_mapping_using_folium

In [59]:
# a simple folium map
import folium

m = folium.Map(location=[lat,lon])
m

ModuleNotFoundError: No module named 'folium'

In [25]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

NameError: name 'folium' is not defined

In [ ]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

## Save the folium map as an html file

In [ ]:
my_map.save('camp.html')

## Evaluation

<img src="images/yoh.png">
A big thank you to all of you who have come today. Please take a moment to fill this survey, as we always want to hear from you.

https://docs.google.com/forms/d/e/1FAIpQLSdVTXFIt4-8c6NQGJc_qORMKIzEmvmBKWFTgU7Ek6AZEq3Xww/viewform


# Resources

- [Google's Python Tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)
- [Earth Lab](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/)
    